In [ ]:
!pip install -U pip

In [ ]:
!pip install -U dill

In [ ]:
!pip install -U nltk #==3.4.5

In [ ]:
from nltk.util import pad_sequence
from nltk.util import bigrams
from nltk.util import ngrams
from nltk.util import everygrams

## Lets get some real data and tokenize it

In [1]:
source_path = r"input/Десперо/DiCamillo Kate. The Tale of Despereaux - royallib.com.txt"

In [1]:
source_url = r"https://uc1ba2935594911273cb019a979f.dl.dropboxusercontent.com/cd/0/get/AwfcsVbUXmXzUdQqZ-yXILh1SMOtwh969xh-RwWKfPvVaHRsbKBC749OabQS8cn9SkSIMKA8EaV3PQLin_HgQzlzfjIlRaDiKG3rWpP3FQQfVg/file?_download_id=561601465769664458798338779964772424300021697661964057822012315&_notify_domain=www.dropbox.com&dl=1"

In [2]:
from urllib import request

In [6]:
with request.urlopen(source_url) as f:
    source_text_dec = f.read().decode('utf-8')

In [ ]:
with open(source_path, 'rt') as f:
    source_text_dec = f.read()

In [7]:
len(source_text_dec)

182767

In [8]:
source_text_dec[:100]

'The Tale of Despereaux\r\n\r\nby Kate DiCamillo\r\n\r\nFor Luke, who asked for\r\n\r\nthe story of an unlikely h'

In [9]:
from nltk import word_tokenize, sent_tokenize 

In [11]:
# import nltk
# nltk.download('punkt')

In [10]:
sent_tokenized = sent_tokenize(source_text_dec)
len(sent_tokenized)

4234

In [12]:
sent_tokenized[0]

'The Tale of Despereaux\r\n\r\nby Kate DiCamillo\r\n\r\nFor Luke, who asked for\r\n\r\nthe story of an unlikely hero\r\n\r\n\r\n\r\nThe world is dark, and light is precious.'

In [17]:
tokenized = word_tokenize(source_text_dec)
len(tokenized)

42399

In [18]:
len(sent_tokenized)

4234

In [33]:
sent_tokenized[0]

['the',
 'tale',
 'of',
 'despereaux',
 'by',
 'kate',
 'dicamillo',
 'for',
 'luke',
 ',',
 'who',
 'asked',
 'for',
 'the',
 'story',
 'of',
 'an',
 'unlikely',
 'hero',
 'the',
 'world',
 'is',
 'dark',
 ',',
 'and',
 'light',
 'is',
 'precious',
 '.']

In [20]:
tokenized[1]

'Tale'

In [21]:
print(source_text_dec[:100])

The Tale of Despereaux

by Kate DiCamillo

For Luke, who asked for

the story of an unlikely h


In [22]:
# Preprocess the tokenized text for 3-grams language modelling
from nltk.lm.preprocessing import padded_everygram_pipeline
from nltk.lm import MLE

In [ ]:
# Tokenize the text.
sent_tokenized = [list(map(str.lower, tokenized)) 
                  for sent in sent_tokenize(source_text_dec)]

In [ ]:
n = 3
train_data, padded_sents = padded_everygram_pipeline(n, sent_tokenized)

# Training an N-gram Model

Having prepared our data we are ready to start training a model. As a simple example, let us train a Maximum Likelihood Estimator (MLE).

We only need to specify the highest ngram order to instantiate it.

In [ ]:
model = MLE(n) # Lets train a 3-grams model, previously we set n=3

Initializing the MLE model, creates an empty vocabulary

In [ ]:
len(model.vocab)

... which gets filled as we fit the model.

In [ ]:
model.fit(train_data, padded_sents)
print(model.vocab)

In [ ]:
len(model.vocab)

The vocabulary helps us handle words that have not occurred during training.

In [82]:
print(model.vocab.lookup(tokenized_text[1]))

<UNK>


In [69]:
# If we lookup the vocab on unseen sentences not from the training data, 
# it automatically replace words not in the vocabulary with `<UNK>`.
print(model.vocab.lookup('despereaux by kate is never random lah .'.split()))

('despereaux', 'by', 'kate', 'is', '<UNK>', '<UNK>', '<UNK>', '.')


Moreover, in some cases we want to ignore words that we did see during training but that didn't occur frequently enough, to provide us useful information. 

You can tell the vocabulary to ignore such words using the `unk_cutoff` argument for the vocabulary lookup, To find out how that works, check out the docs for the [`nltk.lm.vocabulary.Vocabulary` class](https://github.com/nltk/nltk/blob/develop/nltk/lm/vocabulary.py)

**Note:** For more sophisticated ngram models, take a look at [these objects from `nltk.lm.models`](https://github.com/nltk/nltk/blob/develop/nltk/lm/models.py):

 - `Lidstone`: Provides Lidstone-smoothed scores.
 - `Laplace`: Implements Laplace (add one) smoothing.
 - `InterpolatedLanguageModel`: Logic common to all interpolated language models (Chen & Goodman 1995).
 - `WittenBellInterpolated`: Interpolated version of Witten-Bell smoothing.

# N-gram Language Model analysis

In [75]:
for c in model.counts[3]:
    print(c)

('<s>', '<s>')
('<s>', 'the')
('the', 'tale')
('tale', 'of')
('of', 'despereaux')
('despereaux', 'by')
('by', 'kate')
('kate', 'dicamillo')
('dicamillo', 'for')
('for', 'luke')
('luke', ',')
(',', 'who')
('who', 'asked')
('asked', 'for')
('for', 'the')
('the', 'story')
('story', 'of')
('of', 'an')
('an', 'unlikely')
('unlikely', 'hero')
('hero', 'the')
('the', 'world')
('world', 'is')
('is', 'dark')
('dark', ',')
(',', 'and')
('and', 'light')
('light', 'is')
('is', 'precious')
('precious', '.')
('.', '</s>')


In [ ]:
from nltk.util import ngrams
text_trigrams = [ngrams(sent, 3) for sent in tokenized_text]
text_bigrams = [ngrams(sent, 2) for sent in tokenized_text]
text_unigrams = [ngrams(sent, 1) for sent in tokenized_text]

In [ ]:
from nltk.lm import NgramCounter
ngram_counts = NgramCounter(text_trigrams + text_bigrams + text_unigrams)

In [ ]:
ngram_counts

In [ ]:
ngram_counts[2]

When it comes to ngram models the training boils down to counting up the ngrams from the training corpus.

In [ ]:
print(model.counts)

This provides a convenient interface to access counts for unigrams...

In [ ]:
model.counts['despereaux'] # i.e. Count('language')

...and bigrams for the phrase "language is"

In [ ]:
model.counts[['said']]['despereaux'] # i.e. Count('is'|'language')

... and trigrams for the phrase "language is never"

In [ ]:
model.counts[['language', 'despereaux']]['never'] # i.e. Count('never'|'language is')

And so on. However, the real purpose of training a language model is to have it score how probable words are in certain contexts.

This being MLE, the model returns the item's relative frequency as its score.

In [ ]:
model.score('despereaux') # P('language')

In [ ]:
model.score('said', 'despereaux'.split())  # P('is'|'language')

In [ ]:
model.score('never', 'language is'.split())  # P('never'|'language is')

Items that are not seen during training are mapped to the vocabulary's "unknown label" token.  This is "<UNK>" by default.


In [ ]:
model.score("<UNK>") == model.score("lah")

In [ ]:
model.score("<UNK>") == model.score("leh")

In [ ]:
model.score("<UNK>") == model.score("lor")

To avoid underflow when working with many small score values it makes sense to take their logarithm. 

For convenience this can be done with the `logscore` method.


In [ ]:
model.logscore("never", "language is".split())

# Generation using N-gram Language Model

One cool feature of ngram models is that they can be used to generate text.

In [ ]:
print(model.generate(20, random_seed=7))

We can do some cleaning to the generated tokens to make it human-like.

In [ ]:
from nltk.tokenize.treebank import TreebankWordDetokenizer

detokenize = TreebankWordDetokenizer().detokenize

def generate_sent(model, num_words, random_seed=42):
    """
    :param model: An ngram language model from `nltk.lm.model`.
    :param num_words: Max no. of words to generate.
    :param random_seed: Seed value for random.
    """
    content = []
    for token in model.generate(num_words, random_seed=random_seed):
        if token == '<s>':
            continue
        if token == '</s>':
            break
        content.append(token)
    return detokenize(content)

In [ ]:
generate_sent(model, 20, random_seed=7)

In [ ]:
print(model.generate(28, random_seed=0))

In [ ]:
generate_sent(model, 28, random_seed=0)

In [ ]:
generate_sent(model, 20, random_seed=1)

In [ ]:
generate_sent(model, 20, random_seed=30)

In [ ]:
generate_sent(model, 20, random_seed=42)

# Saving the model 

The native Python's pickle may not save the lambda functions in the  model, so we can use the `dill` library in place of pickle to save and load the language model.


In [ ]:
import dill as pickle 

with open('kilgariff_ngram_model.pkl', 'wb') as fout:
    pickle.dump(model, fout)

In [ ]:
with open('kilgariff_ngram_model.pkl', 'rb') as fin:
    model_loaded = pickle.load(fin)

In [ ]:
generate_sent(model_loaded, 20, random_seed=42)

# Lets try some generating with Donald Trump data!!!


**Dataset:** https://www.kaggle.com/kingburrito666/better-donald-trump-tweets#Donald-Tweets!.csv


In this part, I'll be munging that data as how I would be doing it at work. 
I've really no seen the data before but I hope this session would be helpful for you to see how to approach new datasets with the skills you have.

In [ ]:
import pandas as pd
df = pd.read_csv('input/Donald-Tweets.csv')
df.head()

In [ ]:
trump_corpus = list(df['Tweet_Text'].apply(word_tokenize))

In [ ]:
# Preprocess the tokenized text for 3-grams language modelling
n = 3
train_data, padded_sents = padded_everygram_pipeline(n, trump_corpus)

In [ ]:
from nltk.lm import MLE
trump_model = MLE(n) # Lets train a 3-grams model, previously we set n=3
trump_model.fit(train_data, padded_sents)

In [ ]:
generate_sent(trump_model, num_words=20, random_seed=42)

In [ ]:
generate_sent(trump_model, num_words=10, random_seed=0)

In [ ]:
generate_sent(trump_model, num_words=50, random_seed=10)

In [ ]:
print(generate_sent(trump_model, num_words=100, random_seed=52))